Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_psi_t_var_formula">_psi_t_var_formula</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, b, c, k, m, n, t, P, defaults, Function
from proveit.logic import Equals, InSet
from proveit.numbers import zero, one, two, i, e, pi, Add, Exp, Less, LessEq, Mult, Neg, subtract
from proveit.numbers import Complex, Interval, Natural
from proveit.numbers.exponentiation import (
        add_one_right_in_exp, exp_eq_for_eq_base_and_exp, exponential_monotonocity)
from proveit.numbers.number_sets.natural_numbers import fold_forall_natural_pos
from proveit.physics.quantum.QPE import _phase, _phase_is_real

In [ ]:
%proving _psi_t_var_formula

In [ ]:
defaults.assumptions = _psi_t_var_formula.all_conditions()

In [ ]:
# the induction theorem for positive naturals
fold_forall_natural_pos

In [ ]:
# instantiate the induction theorem
induction_inst = fold_forall_natural_pos.instantiate(
    {Function(P,t):_psi_t_var_formula.instance_expr, m:t, n:t})

### Some Related Properties and Definitions Needed for Later Processing
Mainly: some domains and orderings. Notice that throughout this section devoted to the induction proof, $t$ is a _variable_, not a literal.

In [ ]:
# used when processing products involving the phase phi
_phase_is_real

In [ ]:
two_pow_t_var_less_one = subtract(Exp(two, t), one)

In [ ]:
# two_pow_t_less_one_is_nat_pos

In [ ]:
two_pow_t_var_less_one.deduce_in_number_set(Natural)

In [ ]:
LessEq(zero, two_pow_t_var_less_one).prove()

In [ ]:
Less(t, Add(t, one)).prove()

In [ ]:
exponential_monotonocity

In [ ]:
exponential_monotonocity_inst = exponential_monotonocity.instantiate({a: two, b: t, c: Add(t, one)})

In [ ]:
# Used to allow a splitting of a summation into the sum of two summations
exponential_monotonocity_inst.derive_shifted(Neg(one))

For later summation index manipulations, we want to establish that $2^{t+1}-2^{t} = 2^{t}$ (and more specifically we will need $2^{t+1}-2^{t}-1 = 2^{t}-1$).

In [ ]:
add_one_right_in_exp

In [ ]:
two_to_quant_t_var_plus_1_factored = add_one_right_in_exp.instantiate(
        {a: two, b: t}).derive_reversed()

In [ ]:
index_shift_simplification = two_to_quant_t_var_plus_1_factored.substitution(
        subtract(Exp(two, Add(t, one)), Exp(two, t)))

In [ ]:
# step no longer needed
# index_shift_simplification = index_shift_simplification.inner_expr().rhs.factor(
#         Exp(two, t))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().rhs.simplify()

In [ ]:
index_shift_simplification = index_shift_simplification.right_add_both_sides(Neg(one))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().lhs.commute(init_idx=1, final_idx=2)

In [ ]:
# index_shift_simplification = index_shift_simplification.inner_expr().rhs.with_subtraction_at(1)

### Base Case

In [ ]:
base_case = induction_inst.antecedent.operands[0]

We have $|\psi_{t}\rangle$ defined as a tensor product (the result of the first phase of the quantum circuit, and the LHS of Nielsen & Chuang's identity 5.20 on pg 222):

In [ ]:
# need psi_t_var_def re-defined with a Ket instead of a NumKet on the left
from proveit.physics.quantum.QPE import _psi_t_var_as_tensor_prod
_psi_t_var_as_tensor_prod

For $\psi'_{1}$, we prove a useful equality then instantiate the `psi_t_var_as_tensor_prod` with $t=1$:

In [ ]:
psi_1_def = _psi_t_var_as_tensor_prod.instantiate({t:one})

Then show that the summation formula also gives the same qbit result

In [ ]:
sum_0_to_1 = base_case.rhs

In [ ]:
sum_0_to_1_processed_01 = sum_0_to_1.inner_expr().operands[1].partitioning_first()

In [ ]:
# finish off the Base Case
base_case_jdgmt = sum_0_to_1_processed_01.sub_left_side_into(psi_1_def)

### Inductive Step

In [ ]:
inductive_step = induction_inst.antecedent.operands[1]

In [ ]:
defaults.assumptions = defaults.assumptions + inductive_step.conditions.entries

First, partition the summation:
$\sum_{k=0}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} = \sum_{k=0}^{2^{t}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} + \sum_{k=2^{t}}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1}$

In [ ]:
summation_partition_01 = (
    inductive_step.instance_expr.rhs.operands[1]
    .partitioning(two_pow_t_var_less_one))

Then shift the second summation of that partition, so that the two summations then have the same index domain:

In [ ]:
summation_partition_02 = summation_partition_01.inner_expr().rhs.operands[1].shift(Neg(Exp(two, t)))

In [ ]:
# recall the following simplification engineered earlier in the nb:
index_shift_simplification

In [ ]:
summation_partition_03 = summation_partition_02.inner_expr().rhs.operands[1].domain.upper_bound.substitute(index_shift_simplification)

We want to rewrite the summand of that 2nd summation on the rhs now by (1) expanding the exponential term and (2) rewriting the $|k+2^t{\rangle}_{t+1}$ ket as $|1\rangle \otimes |k{\rangle}_t$. This takes a little work.

In [ ]:
rhs_2nd_sum = summation_partition_03.rhs.operands[1]

In [ ]:
summand_processed_01 = rhs_2nd_sum.summand.inner_expr().operands[0].exponent.distribution(
    4, assumptions=[*defaults.assumptions,
                    InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summand_processed_02 = summand_processed_01.inner_expr().rhs.operands[0].exponent_separate(assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# commute the NumKet num operands to format it for replacement later using the prepend_num_ket_with_one_ket thm
summand_processed_03 = summand_processed_02.inner_expr().rhs.operands[1].num.commute(
    assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
from proveit.physics.quantum.algebra import prepend_num_ket_with_one_ket
prepend_num_ket_with_one_ket

In [ ]:
prepend_num_ket_with_one_ket_inst = prepend_num_ket_with_one_ket.instantiate(
        {n: t, k: k}, assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# can delete this cell later
summand_processed_03

In [ ]:
summand_processed_04 = (
    summand_processed_03.inner_expr().rhs.operands[1]
    .substitute(prepend_num_ket_with_one_ket_inst))

In [ ]:
# old version
# summand_processed_04 = multi_tensor_prod_induct_1_inst.sub_left_side_into(summand_processed_03)

In [ ]:
summand_processed_05 = summand_processed_04.inner_expr().rhs.operands[0].commute(assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# summand_processed_generalized = summand_processed_05.generalize(k, domain=Interval(zero, subtract(Exp(two, t), one)))

In [ ]:
# reminder of summation_partition_03
summation_partition_03

In [ ]:
# Now substitute our summand processing into the larger summations expression
summation_partition_04 = (
    summation_partition_03.inner_expr().rhs.operands[1].summand.substitute(
        summand_processed_05))

$|0\rangle \otimes \sum (e^{2\pi i \varphi k} (|k\rangle_{t})) + (e^{2\pi i \varphi 2^t}|1\rangle)\otimes\sum (e^{2\pi i \varphi k} |k\rangle_t) $

$\left(|0\rangle \otimes e^{2\pi i \varphi 2^t}|1\rangle\right)\sum (e^{2\pi i \varphi k} |k\rangle_t) $

In [ ]:
# checking some types inside the rhs summand above
type(summation_partition_04.rhs.operands[1].summand.scalar)

We also then want to:

(1) pull the tensor product out of the 2nd summation, and

(2) pull the non-$k$-dependent exponential factor out of the 2nd summation.

(|0> + c |1>) Sum()

In [ ]:
# for convenience:
temp_factors = summation_partition_04.rhs.operands[1].summand.operands[0]

In [ ]:
type(temp_factors)

In [ ]:
# for convenience:
temp_factor_01 = temp_factors.operands[0]

In [ ]:
# for convenience:
temp_factor_02 = temp_factors.operands[1]

In [ ]:
# another convenience: an expr for the current domain of index k
k_domain = summation_partition_04.rhs.operands[1].conditions[0]

In [ ]:
from proveit.linear_algebra import VecSpaces
from proveit.numbers import Real
VecSpaces.default_field=Complex
summation_partition_05 = summation_partition_04.inner_expr().rhs.operands[1].factors_extract(
    field=None, assumptions=defaults.assumptions )

In [ ]:
summation_partition_06 = summation_partition_05.inner_expr().rhs.operands[1].scaled.tensor_prod_factor(
    1, assumptions=defaults.assumptions+(k_domain,))

(|0>*b) + ((e*|1>)*b) = (a + c)*b

In [ ]:
type(summation_partition_06.rhs.operands[1])

In [ ]:
from proveit.linear_algebra.tensors import factor_scalar_from_tensor_prod
factor_scalar_from_tensor_prod

In [ ]:
scalar_mult_expr = summation_partition_06.rhs.operands[1]

In [ ]:
from proveit.numbers import num
from proveit import V, K, alpha, i, k, a, b, c, ExprTuple
_V_sub = VecSpaces.known_vec_space(scalar_mult_expr)
_K_sub = VecSpaces.known_field(_V_sub)
_alpha_sub = scalar_mult_expr.scalar
_a_sub = ()
_b_sub = scalar_mult_expr.scaled.operands[0]
_c_sub = ExprTuple(scalar_mult_expr.scaled.operands[1])
_i_sub = num(len(_a_sub))
_k_sub = one
impl = factor_scalar_from_tensor_prod.instantiate(
        {V: _V_sub, K:_K_sub, alpha:_alpha_sub, i:_i_sub,
         k:_k_sub, a:_a_sub, b:_b_sub, c:_c_sub})

In [ ]:
_V_sub = VecSpaces.known_vec_space(scalar_mult_expr)

factor_scalar_from_tensor_prod.instantiate(
    {})

In [ ]:
# from proveit.linear_algebra import VecSpaces
# VecSpaces.default_field=Complex
# summation_partition_04.inner_expr().rhs.operands[1].tensor_prod_factor(
#     1, field=Complex, assumptions=defaults.assumptions+(k_domain,) )

In [ ]:
tensor_prod_linearity

In [ ]:
tensor_prod_sub = tensor_prod_linearity.instantiate(
        {j: k, a: Ket(one), b: RegisterKet(j, t), c: zero,
         d: subtract(Exp(two, t), one), t: t, fk: temp_factors},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_partition_shifted_03 = tensor_prod_sub.sub_right_side_into(summation_partition_shifted_02)

In [ ]:
# old
# summation_split_shifted = tensor_prod_sub.sub_right_side_into(summation_split_shifted)

In [ ]:
distributive_summation_spec

In [ ]:
distributive_summation_spec_inst = distributive_summation_spec.instantiate(
        {j:k, c:zero, d:subtract(Exp(two, t), one), x:temp_factor_01, gk:temp_factor_02, fk:RegisterKet(k, t)})

In [ ]:
summation_partition_shifted_04 = distributive_summation_spec_inst.sub_right_side_into(summation_partition_shifted_03)

Now we want to effect a substitution into the first summation on the rhs, taking $|k{\rangle}_{t+1}$ to $|0\rangle{}\otimes|k{\rangle}_{t}$. As with the earlier effort inside the 2nd summation, this is somewhat challenging because the replacement involves an expression containing the index $k$ and thus eventually requires an `instance_substitute` step.

In [ ]:
# our axiom/theorem to apply (in reverse)
multi_tensor_prod_induct_0

In [ ]:
# our axiom/theorem instantiated
multi_tensor_prod_induct_0_inst_reversed = multi_tensor_prod_induct_0.instantiate(
        {t: t, k: k},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))]).derive_reversed()

In [ ]:
first_summand = summation_partition_shifted_04.rhs.operands[0].summand

In [ ]:
# old
# first_summand = summation_split_shifted.rhs.operands[0].summand

In [ ]:
first_summand_judgment = multi_tensor_prod_induct_0_inst_reversed.substitution(first_summand.inner_expr().operands[1])

In [ ]:
first_summand_judgment_gen = first_summand_judgment.generalize(k, domain=Interval(zero, subtract(Exp(two, t), one)))

In [ ]:
summation_partition_shifted_05 = summation_partition_shifted_04.inner_expr().rhs.operands[0].instance_substitute(first_summand_judgment_gen)

In [ ]:
# old
# summation_split_shifted = summation_split_shifted.inner_expr().rhs.operands[0].instance_substitute(first_summand_judgment_gen)

Next we want to pull the $|0\rangle$ out of the first summation, so again we use tensor_prod_linearity:

In [ ]:
tensor_prod_linearity

In [ ]:
tensor_prod_sub = tensor_prod_linearity.instantiate(
        {j: k, a: Ket(zero), b: RegisterKet(j, t), c: zero,
         d: subtract(Exp(two, t), one), t: t, fk: temp_factor_02},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_partition_shifted_06 = tensor_prod_sub.sub_right_side_into(summation_partition_shifted_05)

In [ ]:
# old
# summation_split_shifted = tensor_prod_sub.sub_right_side_into(summation_split_shifted)

We need a few more manipulations of that second term on the rhs: pulling the exponential factor out to the front and reassociating.

In [ ]:
factor_complex_scalar_from_tensor_prod

In [ ]:
# for convenience, name that 2nd summation on the rhs
the_summation_factor = summation_partition_shifted_06.rhs.operands[1].operands[1].operands[1]

In [ ]:
temp_factored_tensor_prod = factor_complex_scalar_from_tensor_prod.instantiate(
        {m: one, n: zero, alpha: temp_factor_01, x: (Ket(one),), y: the_summation_factor,
        z:()},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_partition_shifted_07 = temp_factored_tensor_prod.sub_right_side_into(summation_partition_shifted_06)

In [ ]:
# old
# summation_split_shifted = temp_factored_tensor_prod.sub_right_side_into(summation_split_shifted)

In [ ]:
# instead probably use factor_scalar_from_tensor_prod
scalar_tensor_associativity

In [ ]:
scalar_tensor_associativity_inst = scalar_tensor_associativity.instantiate(
        {alpha: temp_factor_01, x: Ket(one), y: the_summation_factor})

In [ ]:
summation_partition_shifted_08 = scalar_tensor_associativity_inst.sub_right_side_into(summation_partition_shifted_07)

In [ ]:
# old
# summation_split_shifted = scalar_tensor_associativity_inst.sub_right_side_into(summation_split_shifted)

In [ ]:
distribute_tensor_prod_over_sum

In [ ]:
from proveit import Variable
i_var = Variable('i')

In [ ]:
distribute_tensor_prod_over_sum_inst = distribute_tensor_prod_over_sum.instantiate(
        {i_var: zero, j: two, k: one, y: (Ket(zero), Mult(temp_factor_01, Ket(one))), z: (the_summation_factor, )},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_partition_shifted_09 = distribute_tensor_prod_over_sum_inst.sub_left_side_into(summation_partition_shifted_08)

In [ ]:
# old
# summation_split_shifted = distribute_tensor_prod_over_sum_inst.sub_left_side_into(summation_split_shifted)

In [ ]:
p_t_def

In [ ]:
p_t_def.instantiate()

In [ ]:
p_prime_r_def

In [ ]:
p_prime_t = p_prime_r_def.instantiate({r:t})

In [ ]:
summation_partition_shifted_10 = p_prime_t.sub_left_side_into(summation_partition_shifted_09)

In [ ]:
# old
# summation_split_shifted = p_prime_t.sub_left_side_into(summation_split_shifted)

In [ ]:
psi_t_var_formula

SOME TESTING HERE

In [ ]:
test_frac = frac(one, Exp(two, frac(Add(t, one), two)))

In [ ]:
test_frac_02 = test_frac.inner_expr().denominator.exponent.distribution()

In [ ]:
test_frac_03 = test_frac_02.inner_expr().rhs.denominator.exponent_separate()

In [ ]:
test_frac_04 = test_frac_03.inner_expr().rhs.factor(frac(one, Exp(two, frac(one, two))))

In [ ]:
from proveit.linear_algebra import TensorProd
test_tensor = Mult(test_frac, TensorProd(a, b))

In [ ]:
test_tensor_02 = test_tensor.inner_expr().factors[0].substitution(test_frac_04)

In [ ]:
test_tensor_02.inner_expr().rhs.disassociate(0)

In [ ]:
scalar_tensor_associativity

In [ ]:
factor_complex_scalar_from_tensor_prod

TESTING ENDS HERE

In [ ]:
# hmmm, now we're in trouble here …
# psi_prime_t_formula

In [ ]:
# Recall our inductive hypothesis:
for item in defaults.assumptions:
    if isinstance(item, Equals):
        inductive_hypothesis = item
inductive_hypothesis

In [ ]:
summation_split_shifted = inductive_hypothesis.sub_left_side_into(summation_split_shifted)

In [ ]:
psi_prime_expansion_inst = psi_prime_expansion.instantiate({t: t})

In [ ]:
psi_prime_t_plus_1_formula = psi_prime_expansion_inst.sub_left_side_into(summation_split_shifted).derive_reversed()

In [ ]:
assert False

OLD below!

In [ ]:
# this is the literal t_ representing the size of the first register
t_in_natural_pos

In [ ]:
psi_prime_t_def

In [ ]:
# instantiate the var t to the literal t_ representing the size of the first register
psi_prime_t_def_inst = psi_prime_t_def.instantiate({t:t_})

In [ ]:
psi_t_lit_def

In [ ]:
psi_t_in_terms_of_psi_prime = psi_prime_t_def_inst.sub_left_side_into(psi_t_lit_def)

In [ ]:
psi_prime_t_lit_formula

In [ ]:
psi_prime_t_lit_formula.sub_right_side_into(psi_t_in_terms_of_psi_prime)

In [ ]:
# %qed